Test models on AWS bedrock using boto3

In [1]:
import os
import json
import boto3
import openai

In [2]:
with open("../config.json", "r") as f:
    CONFIG = json.load(f)

In [ ]:
client = boto3.client(
    'bedrock-runtime', 
    aws_access_key_id = CONFIG["aws_access_key_id"], 
    aws_secret_access_key = CONFIG["aws_secret_access_key"],
    region_name = "us-east-1"
)

In [ ]:
sample_prompt = '''
SYSTEM: Provide a response for the tutor to the student learning Japanese.
Provide only the succeeding response. Do not continue the conversation further.
Separate the output into two parts: 
<feedback> Give feedback on the grammar of the student in his last response.
<response> A response to keep the conversation going.
Use this json syntax:
{
"feedback": "<feedback>",
"response": "<response>"
}
The student is a native english speaker. Comment on his grammar in English, but write samples in Japanese text.
The response should be in Japanese text.

STUDENT: 今日、僕は何が勉強しますか？
TUTOR:
'''

In [ ]:
model_id = 'cohere.command-text-v14'
accept = '*/*'
content_type = 'application/json'


# NOTE: these parameters need to be appropriate for the model
body = json.dumps({
    "prompt": sample_prompt,
    "max_tokens": 500,
    "temperature": 0.0,
})

response = client.invoke_model(
    modelId = model_id,
    body = body,
    accept = accept,
    contentType = content_type
)

response_body = json.loads(response.get('body').read())
print(response_body['generations'][0]['text'])

Looks like the Cohere model got the syntax right, but the feedback is actually wrong..
<br> Could get better with reference doc, but check if other models get this right just from weights.

In [3]:
openai.api_key = CONFIG["openai_api_key"]

In [4]:
conversation = [
    {
        "role": "system", 
        "content": '''
        You are a tutor to a student learning Japanese.
        Converse with the student and give feedback on his Japanese.
        Your response should contain both feedback for the student and a response to continue the conversation.
        Respond using this json syntax:
        {
            "feedback": "<feedback>",
            "message": "<message>"
        }
        In the <feedback> value, provide feedback and suggestions for improving his Japanese.
        The student is a native english speaker. 
        Comment on his grammar in English, but write samples in Japanese text.
        
        In the <message> value, reply to his query to keep the conversation going.
        The message should be in Japanese text.
        Make a message that encourages him to continue conversing.

        Here is a sample response:
        {
            "feedback": "
            In your previous statement, it is more correct to say '教えてください'　than '教えるください'.
            ",
            "message": "今日は何をしましたか？"
        }
        '''
    },
    {"role": "user", "content": "今日、僕は何が勉強しますか？"},
]

In [5]:
# gpt-3.5 fails to separate the feedback and the message?
    # maybe I can have them as separate generations?
    # but keep trying to modify the prompt.

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = conversation,
    max_tokens = 500,
    temperature = 0.0,
)
print(response.choices[0].message.content)

「今日、僕は何を勉強しますか？」という質問は文法的に正しいです。ただし、もう少し自然な表現にするためには、「今日は何を勉強する予定ですか？」と言うと良いでしょう。

それでは、何を勉強する予定ですか？どの科目に興味がありますか？


Try to separate the response part and the feedback part into 2 calls
<br> Use openAI for now

In [7]:
system_prompt_buddy = '''
You are a practice buddy to a student learning Japanese.
Converse with the student.
The student is a native English speaker.
If the student responds in another language, encourage him to chat in Japanese.
You should only respond in Japanese.
Use grammar and vocabulary appropriate for JLPT N4 level.
Use both formal and casual Japanese so the student can learn both.
If necessary, ask questions to keep the conversation going.
'''

In [9]:
query1 = "こんにちは！今から日本語の練習します。"

conversation = [
    {
        "role": "system", 
        "content": system_prompt_buddy
    },
    {"role": "user", "content": query1},
]

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = conversation,
    max_tokens = 500,
    temperature = 0.0,
)
print(response.choices[0].message.content)
res_txt1 = response.choices[0].message.content

こんにちは！日本語の練習、頑張りましょう！どんなことを練習したいですか？


In [10]:
query2 = "今日は土曜日だよ。たくさんねました。11時に起きました。"

conversation = [
    {"role": "system", "content": system_prompt_buddy},
    {"role": "user", "content": query1},
    {"role": "assistant", "content": res_txt1},
    {"role": "user", "content": query2},
]

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = conversation,
    max_tokens = 500,
    temperature = 0.0,
)
print(response.choices[0].message.content)
res_txt2 = response.choices[0].message.content

そうですか、土曜日はゆっくり寝ることができる日ですね。11時に起きたんですね。何時間寝たんですか？


In [11]:
system_prompt_critic = '''
You are an assistant to a student learning Japanese.
A conversation will be provided between the student and the practice buddy.
Instead of responding to the conversation, provide feedback on the grammar, vocabulary and overall language of the student.
The student is a native English speaker.
Provide feedback in English.
Provide feedback that is appropriate for a JLPT N4 level.

Focus on providing feedback on the latest response of the student.
Avoid referring to previous responses unless necessary.
'''

In [13]:
def conversation_to_txt(conversation, **kwargs):
    '''
    Convert a conversation to text which can be put into a single prompt.
    NOTE: In Langchain, we will have to do this from the chat history / memory.
    '''

    conversation_str = ""

    for response in conversation:
        if response["role"] == "system":
            continue

        elif response["role"] == "user":
            conversation_str += "STUDENT: " + response["content"] + "\n"

        elif response["role"] == "assistant":
            conversation_str += "PRACTICE BUDDY: " + response["content"] + "\n"
    
    return conversation_str

conversation_str = conversation_to_txt(conversation)
print(conversation_str)

STUDENT: こんにちは！今から日本語の練習します。
PRACTICE BUDDY: こんにちは！日本語の練習、頑張りましょう！どんなことを練習したいですか？
STUDENT: 今日は土曜日だよ。たくさんねました。11時に起きました。



In [17]:
# gpt-3.5 gets the instruction right, but totally hallucinates.
    # probably need to collect some references for n5 and n4
    # then build a chain/tool/retriever to retrieve references
        # but still okay to build this chain first then modify it later to retrieve references

# gpt-4 actually gives more accurate feedback
    # but probably don't want to rely on that coz it's expensive
    # and also might get things wrong when we get to more complicated levels, grammar.

query = '''
Provide feedback for the student in the following conversation:
{conversation_str}
'''.format(conversation_str = conversation_str)

conversation_critic = [
    {"role": "system", "content": system_prompt_critic},
    {"role": "system", "content": query},
]

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = conversation_critic,
    max_tokens = 500,
    temperature = 0.0,
)
print(response.choices[0].message.content)

Overall, your response is good. Here are a few points to consider:

1. In the sentence "たくさんねました", the verb "ねました" should be changed to "ねました" to match the past tense of the verb "ねる" (to sleep).

2. Instead of saying "今日は土曜日だよ", it would be more natural to say "今日は土曜日です" to indicate that today is Saturday.

3. In the sentence "11時に起きました", it would be more accurate to say "11時に起きました" to indicate that you woke up at 11 o'clock.

Keep up the good work!
